In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


# Param

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Data

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [5]:
np.array(lines).shape

(170652,)

In [6]:
lines[0]

'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)'

- Processing data

In [7]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, attribution = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    # Get unique characters
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
np.array(input_texts).shape

(20000,)

In [9]:
input_texts[0]

'Go.'

In [10]:
np.array(target_texts).shape

(20000,)

In [11]:
target_texts[0]

'\tVa !\n'

In [12]:
input_characters

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [13]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f'}

In [14]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [15]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output t0okens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 74
Number of unique output t0okens: 101
Max sequence length for inputs: 18
Max sequence length for outputs: 59


- Build character dictionary

In [16]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [17]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'X': 45,
 'Y': 46,
 'Z': 47,
 'a': 48,
 'b': 49,
 'c': 50,
 'd': 51,
 'e': 52,
 'f': 53,
 'g': 54,
 'h': 55,
 'i': 56,
 'j': 57,
 'k': 58,
 'l': 59,
 'm': 60,
 'n': 61,
 'o': 62,
 'p': 63,
 'q': 64,
 'r': 65,
 's': 66,
 't': 67,
 'u': 68,
 'v': 69,
 'w': 70,
 'x': 71,
 'y': 72,
 'z': 73}

In [18]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'X': 48,
 'Y': 49,
 'a': 50,
 'b': 51,
 'c': 52,
 'd': 53,
 'e': 54,
 'f': 55,
 'g': 56,
 'h': 57,
 'i': 58,
 'j': 59,
 'k': 60,
 'l': 61,
 'm': 62,
 'n': 63,
 'o': 64,
 'p': 65,
 'q': 66,
 'r': 67,
 's': 68,
 't': 69,
 'u': 70,
 'v': 71,
 'w': 72,
 'x': 73,
 'y': 74,
 'z': 75,
 '\xa0': 76,
 '«': 77,
 '»': 78,
 'À': 79,
 'Ç': 80,
 'É': 81,
 'Ê': 82,
 'Ô': 83,
 'à': 84,
 'â': 85,
 'ç': 86,
 'è': 87,
 'é': 88,
 'ê': 89,
 'ë': 90,
 'î': 91,
 'ï': 92,
 'ô': 93,
 'ù': 94,
 'û': 95,
 'œ': 96,
 '\u2009': 97,
 '‘': 98,
 '’': 99,


# Build encoder & decoder input

- Init input

In [19]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

- One-hot vector

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

- Print encoder_input_data

In [21]:
input_texts[0]

'Go.'

In [22]:
# ' ': index 0
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
# G: Index 26
encoder_input_data[0][0][26]

0.0

In [24]:
# o: Index 58
encoder_input_data[0][1][58]

0.0

- Print decoder_input_data

In [25]:
# \t: start, \n: end
target_texts[0]

'\tVa !\n'

In [26]:
# ' ': index 2 
decoder_input_data[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [27]:
# V: Index 43
decoder_input_data[0][1][43]

0.0

In [28]:
# ' ': index 2 
decoder_target_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [29]:
# V: Index 43
decoder_target_data[0][0][43]

0.0

# Encode & Decoder (implementation of sequence-to-sequence)

- Return_state & return_sequences param of LSTM: https://www.dlology.com/blog/how-to-use-return_state-or-return_sequences-in-keras/
- Encoder states (context vector): initial hidden state of decoder
- Teacher Forcing: https://towardsdatascience.com/what-is-teacher-forcing-3da6217fed1c

- Encoder

In [30]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c] # Context vector of encoder

- Decoder 

In [31]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

- Model

In [32]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train 

In [33]:
import keras.backend.tensorflow_backend as K

In [34]:
with K.tf.device('/gpu:1'):
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/100
16000/16000 [==============================] - 21s 1ms/step - loss: 1.0989 - accuracy: 0.7248 - val_loss: 0.9784 - val_accuracy: 0.7297
Epoch 2/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.7200 - accuracy: 0.7940 - val_loss: 0.7808 - val_accuracy: 0.7700
Epoch 3/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6067 - accuracy: 0.8220 - val_loss: 0.6930 - val_accuracy: 0.7960
Epoch 4/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.5471 - accuracy: 0.8384 - val_loss: 0.6412 - val_accuracy: 0.8126
Epoch 5/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.4981 - accuracy: 0.8519 - val_loss: 0.6020 - val_accuracy: 0.8218
Epoch 6/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.4610 - accuracy: 0.8620 - val_loss: 0.5620 - val_accuracy: 0.8334
Epoch 7/100
16000/16000 [==============================] - 20s 

Epoch 56/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0993 - accuracy: 0.9681 - val_loss: 0.6195 - val_accuracy: 0.8655
Epoch 57/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0973 - accuracy: 0.9687 - val_loss: 0.6270 - val_accuracy: 0.8651
Epoch 58/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0952 - accuracy: 0.9694 - val_loss: 0.6284 - val_accuracy: 0.8653
Epoch 59/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0933 - accuracy: 0.9699 - val_loss: 0.6361 - val_accuracy: 0.8648
Epoch 60/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0914 - accuracy: 0.9705 - val_loss: 0.6417 - val_accuracy: 0.8643
Epoch 61/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0898 - accuracy: 0.9709 - val_loss: 0.6443 - val_accuracy: 0.8644
Epoch 62/100
16000/16000 [==============================] - 20s 1ms/step - loss: 0.0877 - accuracy: 0.9714

In [35]:
# Save model
model.save('s2s.h5')

# Test

In [36]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [37]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [38]:
reverse_input_char_index

{0: ' ',
 1: '!',
 2: '"',
 3: '$',
 4: '%',
 5: '&',
 6: "'",
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '4',
 15: '5',
 16: '6',
 17: '7',
 18: '8',
 19: '9',
 20: ':',
 21: '?',
 22: 'A',
 23: 'B',
 24: 'C',
 25: 'D',
 26: 'E',
 27: 'F',
 28: 'G',
 29: 'H',
 30: 'I',
 31: 'J',
 32: 'K',
 33: 'L',
 34: 'M',
 35: 'N',
 36: 'O',
 37: 'P',
 38: 'Q',
 39: 'R',
 40: 'S',
 41: 'T',
 42: 'U',
 43: 'V',
 44: 'W',
 45: 'X',
 46: 'Y',
 47: 'Z',
 48: 'a',
 49: 'b',
 50: 'c',
 51: 'd',
 52: 'e',
 53: 'f',
 54: 'g',
 55: 'h',
 56: 'i',
 57: 'j',
 58: 'k',
 59: 'l',
 60: 'm',
 61: 'n',
 62: 'o',
 63: 'p',
 64: 'q',
 65: 'r',
 66: 's',
 67: 't',
 68: 'u',
 69: 'v',
 70: 'w',
 71: 'x',
 72: 'y',
 73: 'z'}

In [39]:
reverse_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: '$',
 6: '%',
 7: '&',
 8: "'",
 9: '(',
 10: ')',
 11: ',',
 12: '-',
 13: '.',
 14: '0',
 15: '1',
 16: '2',
 17: '3',
 18: '4',
 19: '5',
 20: '6',
 21: '7',
 22: '8',
 23: '9',
 24: ':',
 25: '?',
 26: 'A',
 27: 'B',
 28: 'C',
 29: 'D',
 30: 'E',
 31: 'F',
 32: 'G',
 33: 'H',
 34: 'I',
 35: 'J',
 36: 'K',
 37: 'L',
 38: 'M',
 39: 'N',
 40: 'O',
 41: 'P',
 42: 'Q',
 43: 'R',
 44: 'S',
 45: 'T',
 46: 'U',
 47: 'V',
 48: 'X',
 49: 'Y',
 50: 'a',
 51: 'b',
 52: 'c',
 53: 'd',
 54: 'e',
 55: 'f',
 56: 'g',
 57: 'h',
 58: 'i',
 59: 'j',
 60: 'k',
 61: 'l',
 62: 'm',
 63: 'n',
 64: 'o',
 65: 'p',
 66: 'q',
 67: 'r',
 68: 's',
 69: 't',
 70: 'u',
 71: 'v',
 72: 'w',
 73: 'x',
 74: 'y',
 75: 'z',
 76: '\xa0',
 77: '«',
 78: '»',
 79: 'À',
 80: 'Ç',
 81: 'É',
 82: 'Ê',
 83: 'Ô',
 84: 'à',
 85: 'â',
 86: 'ç',
 87: 'è',
 88: 'é',
 89: 'ê',
 90: 'ë',
 91: 'î',
 92: 'ï',
 93: 'ô',
 94: 'ù',
 95: 'û',
 96: 'œ',
 97: '\u2009',
 98: '‘',
 99: '’',


In [40]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [41]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Salut !

-